In [1]:
# Import necessary libraries
import random
import numpy as np
from datetime import datetime, timedelta
import pandas as pd


from sklearn.metrics import mean_absolute_error, mean_squared_error



In [2]:

# Set the seed for reproducibility
random.seed(42)
np.random.seed(42)

# Function to ensure time is in HH:MM:SS format
def convert_to_hh_mm_ss(time_str):
    return f"00:{time_str}" if len(time_str.split(':')) == 2 else time_str

# Function to convert timedelta to minutes and seconds format
def timedelta_to_min_sec(td):
    total_seconds = td.total_seconds()
    return f"{int(total_seconds // 60):02}:{int(total_seconds % 60):02}"

# Define the simulation start and end times
start_time = datetime(2023, 1, 1,  8,  0)
end_time = datetime(2024, 1, 1, 16, 0)

# Initialize customer data with various attributes
customers = [{'id': i+1,
              'lambda_weekday': 224/8/500 + np.random.exponential(scale=0.0001),
              'lambda_weekend': 100/8/500 + np.random.exponential(scale=0.00001),
              'tolerance': timedelta(minutes=np.random.uniform(2, 10)),
              'next_possible_call_time': start_time} for i in range(500)]

# Initialize agent data with responsiveness and availability times
agents_20 = [{'id': i+1,
             'responsiveness': min(1, 0.93 + np.random.exponential(scale=0.01)),
             'available_until': start_time} for i in range(20)]

# Function to generate call durations with the possibility of an outlier
def truncated_exponential_with_outlier_inverse(lambda_scale, min_val=3*60, max_val=15*60, outlier_val=25*60, outlier_prob=0.05):
    # Check for the chance of an outlier
    if np.random.random() < outlier_prob:
        return outlier_val

    # CDF values for the truncation limits
    cdf_min = 1 - np.exp(-min_val / lambda_scale)
    cdf_max = 1 - np.exp(-max_val / lambda_scale)

    # Generate a random number between cdf_min and cdf_max
    u = np.random.uniform(cdf_min, cdf_max)

    # Apply the inverse CDF to get the sampled value
    sampled_value = -lambda_scale * np.log(1 - u)

    return sampled_value

# Function to simulate calls for customers based on a NHPP
def simulate_calls_for_NHPP(customer, start_time, end_time):
    calls = []
    while True:
        # Determine call rate based on whether it is a weekday or weekend
        lambda_val = customer['lambda_weekend'] if start_time.weekday() == 6 else customer['lambda_weekday']
        # Generate next call time based on the Poisson process
        next_call_time = start_time + timedelta(hours=np.random.exponential(1 / lambda_val))

        # Add the call to the list if it is within the simulation period
        if next_call_time <= end_time:
            calls.append({'customer_id': customer['id'], 'call_time': next_call_time, 'tolerance': customer['tolerance']})
            start_time = next_call_time
        else:
            break
    return calls

# Initialize the logs for calls, answered calls, and wait times
call_logs, answered_calls_corrected, wait_times_corrected = [], [], []

# Simulate call process for each day within the simulation period
current_time = start_time
while current_time <= end_time:
    # Define the end of the working day
    end_of_day = datetime(current_time.year, current_time.month, current_time.day, 16, 0)
    # Generate call logs for each customer for the day
    call_logs.extend([call for customer in customers for call in simulate_calls_for_NHPP(customer, current_time, end_of_day)])

    # Sort calls by call time
    call_logs.sort(key=lambda x: x['call_time'])
    # Process each call
    for call in call_logs:
        # Filter available agents based on call time and sort by availability
        available_agents = sorted([agent for agent in agents_20 if agent['available_until'] <= call['call_time']], key=lambda x: x['available_until'])

        # If no agents are available or agent does not respond, log the call as abandoned
        if not available_agents or random.random() > available_agents[0]['responsiveness']:
            answered_calls_corrected.append({'customer_id': call['customer_id'], 'call_time': call['call_time'], 'status': 'abandoned', 'agent_id': 'NA', 'call_duration': '00:00', 'wait_time': timedelta(seconds=0)})
            continue

        # Assign the first available agent to the call
        agent = available_agents[0]
        # Calculate wait time and ensure it does not exceed customer tolerance
        wait_time = min(call['tolerance'], call['call_time'] - agent['available_until'])

        # Determine call duration based on a truncated exponential distribution
        call_duration_seconds = truncated_exponential_with_outlier_inverse(3*60)
        # Update the agent's availability based on call duration
        agent['available_until'] = call['call_time'] + timedelta(seconds=call_duration_seconds)

        # Log the answered call with details
        answered_calls_corrected.append({'customer_id': call['customer_id'], 'call_time': call['call_time'], 'status': 'answered', 'agent_id': agent['id'], 'call_duration': f"{int(call_duration_seconds // 60):02}:{int(call_duration_seconds % 60):02}", 'wait_time': wait_time})
        # Add corrected wait time to the list
        wait_times_corrected.append(wait_time)

    # Clear the call logs for the day
    call_logs.clear()
    # Move to the next day and set the starting time
    current_time += timedelta(days=1)
    current_time = datetime(current_time.year, current_time.month, current_time.day, 8, 0)

# Create a DataFrame from the corrected answered call logs
df = pd.DataFrame(answered_calls_corrected)
# Apply conversion of timedelta to minutes and seconds for wait time
df['wait_time'] = df['wait_time'].apply(timedelta_to_min_sec)
# Extract date and time from the call timestamp
df['date'] = df['call_time'].dt.date
df['time'] = df['call_time'].dt.time
# Format time to HH:MM:SS
df['time'] = df['time'].apply(lambda x: x.strftime('%H:%M:%S'))

# Drop the original call_time column
df.drop(columns=['call_time'], inplace=True)
# Display the first 20 rows of the DataFrame
df.head(20)


,customer_id,status,agent_id,call_duration,wait_time,date,time
0,326,answered,1,04:06,02:24,2023-01-01,08:07:44
1,309,answered,2,07:55,04:08,2023-01-01,08:12:35
2,297,answered,3,04:17,07:46,2023-01-01,08:22:29
3,181,answered,4,08:01,03:49,2023-01-01,08:26:36
4,187,answered,5,03:33,07:37,2023-01-01,08:29:15
5,183,answered,6,07:59,06:09,2023-01-01,08:46:29
6,234,answered,7,03:48,02:24,2023-01-01,08:50:47
7,402,answered,8,04:15,08:09,2023-01-01,08:52:56
8,365,answered,9,06:59,03:50,2023-01-01,09:03:41
9,452,answered,10,05:49,08:59,2023-01-01,09:06:05


In [3]:

df= df.copy()

def mmss_to_seconds(mmss):
    if mmss == "N/A":
        return np.nan
    minutes, seconds = map(int, mmss.split(":"))
    return 60 * minutes + seconds

# Use .loc to modify the DataFrame
df.loc[:, 'wait_time_seconds'] = df['wait_time'].apply(mmss_to_seconds)
df.loc[:, 'call_duration_seconds'] = df['call_duration'].apply(mmss_to_seconds)
#df.loc[:, 'answer_speed_seconds'] = df['answer_speed'].apply(mmss_to_seconds)

daily_summary = df.groupby('date').agg(
    Incoming_calls=pd.NamedAgg(column='customer_id', aggfunc='size'),
    Answered_calls=pd.NamedAgg(column='status', aggfunc=lambda x: (x == 'answered').sum()),
    Abandoned_calls=pd.NamedAgg(column='status', aggfunc=lambda x: (x == 'abandoned').sum()),
    Average_wait_time=pd.NamedAgg(column='wait_time_seconds', aggfunc='mean'),
    Average_call_duration=pd.NamedAgg(column='call_duration_seconds', aggfunc='mean'),
   # Average_answer_speed=pd.NamedAgg(column='answer_speed_seconds', aggfunc='mean')
)
daily_summary['Answer_rate'] = daily_summary['Answered_calls'] / daily_summary['Incoming_calls']

daily_summary['Average_wait_time'] = daily_summary['Average_wait_time'].apply(lambda x: timedelta_to_min_sec(timedelta(seconds=x)))
daily_summary['Average_call_duration'] = daily_summary['Average_call_duration'].apply(lambda x: timedelta_to_min_sec(timedelta(seconds=x)))
#daily_summary['Average_answer_speed'] = daily_summary['Average_answer_speed'].apply(lambda x: timedelta_to_min_sec(timedelta(seconds=x)))

daily_summary.reset_index(inplace=True)
daily_summary.head(20)


,date,Incoming_calls,Answered_calls,Abandoned_calls,Average_wait_time,Average_call_duration,Answer_rate
0,2023-01-01,122,114,8,05:32,06:22,0.934426
1,2023-01-02,222,206,16,05:20,05:49,0.927928
2,2023-01-03,248,231,17,05:33,06:04,0.931452
3,2023-01-04,233,215,18,05:16,05:52,0.922747
4,2023-01-05,234,217,17,05:35,05:48,0.927350
5,2023-01-06,220,217,3,05:59,06:43,0.986364
6,2023-01-07,233,216,17,05:25,06:08,0.927039
7,2023-01-08,92,86,6,05:36,07:47,0.934783
8,2023-01-09,204,189,15,05:35,06:21,0.926471
9,2023-01-10,236,222,14,05:24,06:23,0.940678


In [4]:
daily_summary = daily_summary[[
    'date', 'Incoming_calls', 'Answered_calls', 'Answer_rate',
    'Abandoned_calls', 'Average_call_duration', 'Average_wait_time'
]]
daily_summary = daily_summary.drop(columns='date')
daily_summary.head(366)

,Incoming_calls,Answered_calls,Answer_rate,Abandoned_calls,Average_call_duration,Average_wait_time
0,122,114,0.934426,8,06:22,05:32
1,222,206,0.927928,16,05:49,05:20
2,248,231,0.931452,17,06:04,05:33
3,233,215,0.922747,18,05:52,05:16
4,234,217,0.927350,17,05:48,05:35
...,...,...,...,...,...,...
361,231,214,0.926407,17,06:13,05:24
362,235,223,0.948936,12,05:44,05:44
363,241,224,0.929461,17,06:48,05:11
364,97,91,0.938144,6,05:57,05:32


In [5]:
from google.colab import drive
drive.mount('/content/drive')
#Load the real data
real_data = pd.read_csv("/content/drive/MyDrive/final project/Code/Call Center Data.csv")


# Add a column for 'Days of the Week' starting with 'Wednesday'
days_of_week = ['Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday']

# Calculate the day of the week for each row based on the index
real_data['Day of the Week'] = np.array(days_of_week * (len(real_data) // len(days_of_week) + 1))[:len(real_data)]

# Show the first few rows of the updated dataset
real_data.head()

Mounted at /content/drive


,Index,Incoming Calls,Answered Calls,Answer Rate,Abandoned Calls,Answer Speed (AVG),Talk Duration (AVG),Waiting Time (AVG),Service Level (20 Seconds),Day of the Week
0,1,217,204,94.01%,13,0:00:17,0:02:14,0:02:45,76.28%,Wednesday
1,2,200,182,91.00%,18,0:00:20,0:02:22,0:06:55,72.73%,Thursday
2,3,216,198,91.67%,18,0:00:18,0:02:38,0:03:50,74.30%,Friday
3,4,155,145,93.55%,10,0:00:15,0:02:29,0:03:12,79.61%,Saturday
4,5,37,37,100.00%,0,0:00:03,0:02:06,0:00:35,97.30%,Sunday


In [6]:
# Extract the last 365 rows from the real data for validation
validation_data = real_data.tail(366).copy()
# Convert 'Answer Rate' column to float by stripping the '%' sign and converting to float
validation_data['Answer Rate'] = validation_data['Answer Rate'].str.rstrip('%').astype('float') / 100

validation_data = validation_data.drop(columns=['Index', 'Answer Speed (AVG)', 'Service Level (20 Seconds)', 'Day of the Week'])

validation_data.head()


,Incoming Calls,Answered Calls,Answer Rate,Abandoned Calls,Talk Duration (AVG),Waiting Time (AVG)
885,191,178,0.9319,13,0:02:51,0:04:08
886,68,68,1.0000,0,0:02:56,0:01:39
887,498,429,0.8614,69,0:03:09,0:04:12
888,358,329,0.9190,29,0:03:03,0:02:15
889,340,302,0.8882,38,0:02:50,0:05:10


In [7]:
daily_summary.describe()

,Incoming_calls,Answered_calls,Answer_rate,Abandoned_calls
count,366.000000,366.000000,366.000000,366.000000
mean,207.150273,194.415301,0.938575,12.734973
std,45.745356,43.020762,0.017588,4.460878
min,81.000000,74.000000,0.870588,2.000000
25%,210.000000,197.000000,0.927397,10.000000
50%,223.000000,209.000000,0.940522,13.000000
75%,233.000000,219.000000,0.950673,15.000000
max,264.000000,251.000000,0.986364,25.000000


In [8]:
validation_data.describe()

,Incoming Calls,Answered Calls,Answer Rate,Abandoned Calls
count,366.000000,366.000000,366.000000,366.000000
mean,213.904372,196.792350,0.937265,17.112022
std,141.998183,116.015353,0.061042,41.876937
min,7.000000,6.000000,0.534600,0.000000
25%,156.000000,147.000000,0.918675,3.000000
50%,202.000000,193.000000,0.956050,8.000000
75%,270.000000,255.750000,0.976500,17.750000
max,1141.000000,644.000000,1.000000,531.000000


In [9]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

def time_to_seconds(time_str):
    if ':' not in str(time_str):
        return float(time_str)
    else:
        time_parts = list(map(int, str(time_str).split(':')))
        if len(time_parts) == 3:
            h, m, s = time_parts
        elif len(time_parts) == 2:
            h = 0
            m, s = time_parts
        else:
            raise ValueError(f"Invalid time format: {time_str}")
        return h * 3600 + m * 60 + s


# Mapping between columns in validation_data and daily_summary
column_mapping = {
    'Total Incoming Calls': ('Incoming Calls', 'Incoming_calls'),
    'Total Answered Calls': ('Answered Calls', 'Answered_calls'),
    'Answer Rate' : ('Answer Rate', 'Answer_rate'),
    'Abandoned Calls' : ('Abandoned Calls', 'Abandoned_calls'),
    'Average Talk Duration' : ('Talk Duration (AVG)', 'Average_call_duration'),
    'Average Waiting time' : ('Waiting Time (AVG)', 'Average_wait_time')
}

results = []
for metric, (column_true, column_pred) in column_mapping.items():
    y_true = validation_data[column_true].apply(time_to_seconds)
    y_pred = daily_summary[column_pred].apply(time_to_seconds)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))


    results.append([metric, mae, rmse])

results_df = pd.DataFrame(results, columns=['Metric', 'MAE', 'RMSE'])
results_df.head(6)






,Metric,MAE,RMSE
0,Total Incoming Calls,103.617486,149.730961
1,Total Answered Calls,93.240437,123.958370
2,Answer Rate,0.045014,0.063052
3,Abandoned Calls,15.114754,42.193608
4,Average Talk Duration,210.278689,212.590296
5,Average Waiting time,167.330601,203.206408
